# 建立文字生成應用程式

你已經在這個課程中見過一些核心概念，例如 prompt（提示詞），甚至有一個叫做「prompt engineering（提示詞工程）」的專門領域。你可以互動的很多工具，例如 ChatGPT、Office 365、Microsoft Power Platform 等，都支援你用 prompt 去完成某些事情。

如果你想將這種體驗加入到自己的應用程式，就需要了解 prompt、completions（完成）、並選擇一個合適的程式庫來配合。這一章就會教你這些內容。

## 介紹

在本章，你會：

- 了解 openai 程式庫及其核心概念。
- 用 openai 建立一個文字生成應用程式。
- 學會如何運用 prompt、temperature（溫度）、tokens（標記）等概念來建立文字生成應用程式。

## 學習目標

完成本課後，你將能夠：

- 解釋什麼是文字生成應用程式。
- 用 openai 建立一個文字生成應用程式。
- 設定你的應用程式，調整 tokens 數量及 temperature，令輸出更有變化。

## 什麼是文字生成應用程式？

一般來說，當你建立一個應用程式時，它會有以下幾種介面：

- 指令式。Console 應用程式就是典型例子，你輸入一個指令，它就執行一個任務。例如 `git` 就是一個指令式應用程式。
- 使用者介面（UI）。有些應用程式有圖形化介面（GUI），你可以點擊按鈕、輸入文字、選擇選項等等。

### Console 和 UI 應用程式的限制

和指令式應用程式比較：

- **有限制**。你不能隨便輸入任何指令，只能用應用程式支援的那些。
- **語言限定**。有些應用程式支援多種語言，但預設都是為某一種語言而設，就算可以加多語言支援。

### 文字生成應用程式的好處

那麼，文字生成應用程式有什麼不同？

在文字生成應用程式中，你有更大彈性，不再受限於一組指令或特定輸入語言。你可以直接用自然語言和應用程式互動。另一個好處是，你其實已經在和一個訓練自龐大資料庫的數據源互動，而傳統應用程式可能只限於資料庫內的內容。

### 我可以用文字生成應用程式做什麼？

你可以做很多事情，例如：

- **聊天機械人**。一個可以回答關於你公司及產品問題的聊天機械人就很適合。
- **助手**。LLM 很擅長總結文字、從文字中找出重點、產生履歷等內容。
- **程式碼助手**。視乎你用的語言模型，你可以建立一個幫你寫程式碼的助手。例如，你可以用 GitHub Copilot 或 ChatGPT 來協助寫程式。

## 我應該如何開始？

你需要找方法和 LLM 整合，通常有以下兩種做法：

- 用 API。你會建立網絡請求，傳送 prompt，然後收到生成的文字。
- 用程式庫。程式庫會幫你包裝 API 呼叫，令使用更簡單。

## 程式庫/SDK

有幾個常用的 LLM 程式庫，例如：

- **openai**，這個程式庫可以輕鬆連接你的模型並傳送 prompt。

另外還有一些更高層次的程式庫，例如：

- **Langchain**。Langchain 很有名，支援 Python。
- **Semantic Kernel**。Semantic Kernel 是 Microsoft 推出的程式庫，支援 C#、Python 和 Java。

## 用 openai 建立第一個應用程式

現在來看看如何建立第一個應用程式，需要哪些程式庫、步驟等等。

### 安裝 openai

  > [!NOTE] 如果你在 Codespaces 或 Devcontainer 執行這個 notebook，可以跳過這一步

市面上有很多程式庫可以和 OpenAI 或 Azure OpenAI 互動。你也可以用不同的程式語言，例如 C#、Python、JavaScript、Java 等等。  
我們這裡選用 `openai` 的 Python 程式庫，所以會用 `pip` 來安裝。

```bash
pip install openai
```

如果你不是在 Codespaces 或 Dev Container 執行這個 notebook，還需要在你的電腦安裝 [Python](https://www.python.org/)。

### 建立資源

如果你還未做，請跟著以下步驟：

- 在 Azure 建立帳戶 <https://azure.microsoft.com/free/>。
- 申請使用 Azure OpenAI。到 <https://learn.microsoft.com/azure/ai-services/openai/overview#how-do-i-get-access-to-azure-openai> 申請存取權限。

  > [!NOTE]
  > 截至目前為止，你需要申請才能使用 Azure OpenAI。

- 建立一個 Azure OpenAI Service 資源。參考這個教學 [建立資源](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource?pivots=web-portal&WT.mc_id=academic-105485-koreyst)。

### 找到 API 金鑰和端點

這時候，你需要告訴 `openai` 程式庫要用哪個 API 金鑰。你可以到 Azure OpenAI 資源的「Keys and Endpoint」部分，複製「Key 1」的值。

  ![Azure Portal 的 Keys and Endpoint 資源頁面](https://learn.microsoft.com/azure/ai-services/openai/media/quickstarts/endpoint.png?WT.mc_id=academic-105485-koreyst)

有了這些資料後，接下來就可以設定程式庫使用它們。

> [!NOTE]
> 最好將 API 金鑰和程式碼分開。你可以用環境變數來做到這一點。
> - 在 .env 檔案中設定環境變數 `AZURE_OPENAI_API_KEY` 為你的 API 金鑰。如果你已經完成本課前面的練習，這部分應該已經設置好。

### 設定 Azure 配置

如果你用的是 Azure OpenAI，可以這樣設定：

```python
client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT')
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']
```

上面我們設定了以下內容：

- `api_key`，這是在 Azure Portal 找到的 API 金鑰。
- `api_version`，你想用的 API 版本。撰寫時最新版本是 `2023-10-01-preview`。
- `azure_endpoint`，API 的端點。在 Azure Portal 你可以在 API 金鑰旁邊找到。

> [!NOTE]
> `os.environ` 是一個讀取環境變數的函數。你可以用它來讀取像 `AZURE_OPENAI_API_KEY` 和 `AZURE_OPENAI_ENDPOINT` 這些環境變數。

## 產生文字

產生文字的方法是用 `chat.completion` 類別。以下是一個例子：

```python
prompt = "Complete the following: Once upon a time there was a"

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": prompt}])
print(completion.choices[0].message.content)
```

在上面的程式碼中，我們建立了一個 completion 物件，傳入想用的模型和 prompt，然後印出生成的文字。

### Chat completions

到目前為止，你已經見過我們用 `Completion` 來產生文字。但其實還有另一個叫 `ChatCompletion` 的類別，更適合用來做聊天機械人。以下是一個例子：

```python
client = AzureOpenAI(
  azure_endpoint = os.environ('AZURE_OPENAI_ENDPOINT'), 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-05-15"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

completion = client.chat.completions.create(model=deployment, messages=[{"role": "user", "content": "Hello world"}])
print(completion.choices[0].message.content)
```

這個功能我們會在之後的章節再詳細介紹。

## 練習 - 你的第一個文字生成應用程式

現在我們已經學會如何設定和配置 Azure OpenAI 服務，是時候動手建立你的第一個文字生成應用程式。請跟著以下步驟來建立你的應用程式：


1. 建立虛擬環境並安裝 openai：

  > [!NOTE] 如果你喺 Codespaces 或 Devcontainer 執行呢個 notebook，呢一步可以唔使做


In [ ]:
# Create virtual environment
! python -m venv venv
# Activate virtual environment
! source venv/bin/activate
# Install openai package
! pip install openai

> [!NOTE]
> 如果你用緊 Windows，請輸入 `venv\Scripts\activate`，唔好用 `source venv/bin/activate`。

> [!NOTE]
> 要搵你嘅 Azure OpenAI 金鑰，可以去 https://portal.azure.com/，搜尋 `Open AI`，揀返你嘅 `Open AI resource`，然後揀 `Keys and Endpoint`，複製 `Key 1` 嘅數值。


1. 建立一個 *app.py* 檔案，並輸入以下程式碼：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

# add your completion code
prompt = "Complete the following: Once upon a time there was a"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages)

# print response
print(completion.choices[0].message.content)

```output
     very unhappy _____.

    Once upon a time there was a very unhappy mermaid.
    ```


## 不同類型的提示，適用於不同用途

你已經學會如何用提示來生成文字。你甚至有一個可以運行的程式，可以修改和變更來生成不同類型的文字。

提示可以用於各種任務。例如：

- **生成某種文字**。例如，你可以生成一首詩、出一些測驗題目等等。
- **查找資訊**。你可以用提示來查找資訊，例如這個例子：「在網頁開發中，CORS 是什麼意思？」。
- **生成程式碼**。你可以用提示來生成程式碼，例如開發一個用來驗證電郵的正則表達式，甚至可以生成整個程式，例如一個網頁應用程式。

## 更實用的例子：食譜生成器

假設你家裡有一些食材，想煮點東西。你需要一個食譜。找食譜的方法可以用搜尋引擎，或者你可以用 LLM 來幫忙。

你可以這樣寫提示：

> 「請給我 5 個用以下食材的菜式食譜：雞肉、薯仔和紅蘿蔔。每個食譜請列出所有用到的食材。」

根據以上提示，你可能會收到類似以下的回覆：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 2 cloves garlic, minced
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 cloves garlic, minced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 2 cloves garlic, minced
- 1 teaspoon dried oregano
```

這個結果很好，我知道可以煮什麼。這時候，有些有用的改進可以是：

- 篩選掉我不喜歡或有敏感的食材。
- 幫我列出購物清單，以防家裡沒有所有食材。

針對以上情況，我們可以加多一句提示：

> 「請移除含有蒜頭的食譜，因為我對蒜頭敏感，並用其他食材代替。另外，請根據我家裡已經有雞肉、薯仔和紅蘿蔔，幫我列出購物清單。」

現在你會得到新的結果：

```output
1. Roasted Chicken and Vegetables: 
Ingredients: 
- 4 chicken thighs
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 2 tablespoons olive oil
- 1 teaspoon dried thyme
- 1 teaspoon dried oregano
- Salt and pepper, to taste

2. Chicken and Potato Stew: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

3. Chicken and Potato Bake: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 1 cup chicken broth
- Salt and pepper, to taste

4. Chicken and Potato Soup: 
Ingredients: 
- 2 tablespoons olive oil
- 1 onion, diced
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 teaspoon dried oregano
- 1 teaspoon dried thyme
- 4 cups chicken broth
- Salt and pepper, to taste

5. Chicken and Potato Hash: 
Ingredients: 
- 2 tablespoons olive oil
- 2 chicken breasts, cut into cubes
- 2 potatoes, cut into cubes
- 2 carrots, cut into cubes
- 1 onion, diced
- 1 teaspoon dried oregano

Shopping List: 
- Olive oil
- Onion
- Thyme
- Oregano
- Salt
- Pepper
```

這就是你的五個食譜，沒有蒜頭，而且還有一份購物清單，考慮到你家裡已經有的食材。


## 練習 - 建立一個食譜產生器

而家我哋已經模擬咗一個情景，依家就寫程式碼去配合呢個示範情景。請跟住以下步驟：

1. 以現有嘅 *app.py* 檔案作為起點
1. 搵到 `prompt` 變數，然後將佢嘅程式碼改成以下內容：


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

prompt = "Show me 5 recipes for a dish with the following ingredients: chicken, potatoes, and carrots. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

如果你而家運行呢段程式碼，你應該會見到類似以下嘅輸出：

```output
-Chicken Stew with Potatoes and Carrots: 3 tablespoons oil, 1 onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 1/2 cups chicken broth, 1/2 cup dry white wine, 2 tablespoons chopped fresh parsley, 2 tablespoons unsalted butter, 1 1/2 pounds boneless, skinless chicken thighs, cut into 1-inch pieces
-Oven-Roasted Chicken with Potatoes and Carrots: 3 tablespoons extra-virgin olive oil, 1 tablespoon Dijon mustard, 1 tablespoon chopped fresh rosemary, 1 tablespoon chopped fresh thyme, 4 cloves garlic, minced, 1 1/2 pounds small red potatoes, quartered, 1 1/2 pounds carrots, quartered lengthwise, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 1 (4-pound) whole chicken
-Chicken, Potato, and Carrot Casserole: cooking spray, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and shredded, 1 potato, peeled and shredded, 1/2 teaspoon dried thyme leaves, 1/4 teaspoon salt, 1/4 teaspoon black pepper, 2 cups fat-free, low-sodium chicken broth, 1 cup frozen peas, 1/4 cup all-purpose flour, 1 cup 2% reduced-fat milk, 1/4 cup grated Parmesan cheese

-One Pot Chicken and Potato Dinner: 2 tablespoons olive oil, 1 pound boneless, skinless chicken thighs, cut into 1-inch pieces, 1 large onion, chopped, 3 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 bay leaf, 1 thyme sprig, 1/2 teaspoon salt, 1/4 teaspoon black pepper, 2 cups chicken broth, 1/2 cup dry white wine

-Chicken, Potato, and Carrot Curry: 1 tablespoon vegetable oil, 1 large onion, chopped, 2 cloves garlic, minced, 1 carrot, peeled and chopped, 1 potato, peeled and chopped, 1 teaspoon ground coriander, 1 teaspoon ground cumin, 1/2 teaspoon ground turmeric, 1/2 teaspoon ground ginger, 1/4 teaspoon cayenne pepper, 2 cups chicken broth, 1/2 cup dry white wine, 1 (15-ounce) can chickpeas, drained and rinsed, 1/2 cup raisins, 1/2 cup chopped fresh cilantro
```

> NOTE，你嘅LLM係非確定性嘅，所以每次運行程式都可能會有唔同嘅結果。

好啦，等我哋睇下可以點樣改善。為咗令程式更靈活，我哋想確保可以隨時改配料同食譜數量，方便之後加減。


In [ ]:
import os
from openai import AzureOpenAI
from dotenv import load_dotenv

# load environment variables from .env file
load_dotenv()

client = AzureOpenAI(
  azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"], 
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  
  api_version = "2023-10-01-preview"
  )

deployment = os.environ['AZURE_OPENAI_DEPLOYMENT']

no_recipes = input("No of recipes (for example, 5: ")

ingredients = input("List of ingredients (for example, chicken, potatoes, and carrots: ")

# interpolate the number of recipes into the prompt an ingredients
prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# print response
print(completion.choices[0].message.content)

測試程式碼運行，可以係咁：

```output
No of recipes (for example, 5: 3
List of ingredients (for example, chicken, potatoes, and carrots: milk,strawberries

-Strawberry milk shake: milk, strawberries, sugar, vanilla extract, ice cubes
-Strawberry shortcake: milk, flour, baking powder, sugar, salt, unsalted butter, strawberries, whipped cream        
-Strawberry milk: milk, strawberries, sugar, vanilla extract
```

### 加入篩選同購物清單，令功能更完善

而家我哋已經有個可以產生食譜嘅應用程式，佢好有彈性，因為用戶可以輸入想要幾多個食譜，仲可以自訂用咩材料。

如果想再提升功能，可以加以下幾樣嘢：

- **篩選材料**。我哋想可以篩走啲唔鍾意或者會敏感嘅材料。要做到呢個改動，可以修改現有嘅 prompt，喺最後加個篩選條件，好似咁：

    ```python
    filter = input("Filter (for example, vegetarian, vegan, or gluten-free: ")

    prompt = f"Show me {no_recipes} recipes for a dish with the following ingredients: {ingredients}. Per recipe, list all the ingredients used, no {filter}"
    ```

    上面我哋喺 prompt 最尾加咗 `{filter}`，同時都會攞用戶輸入嘅 filter 值。

    而家運行程式嘅輸入示例可以係咁：
    
    ```output    
    No of recipes (for example, 5: 3
    List of ingredients (for example, chicken, potatoes, and carrots: onion,milk
    Filter (for example, vegetarian, vegan, or gluten-free: no milk

    1. French Onion Soup

    Ingredients:
    
    -1 large onion, sliced
    -3 cups beef broth
    -1 cup milk
    -6 slices french bread
    -1/4 cup shredded Parmesan cheese
    -1 tablespoon butter
    -1 teaspoon dried thyme
    -1/4 teaspoon salt
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add beef broth, milk, thyme, salt, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Place french bread slices on soup bowls.
    5. Ladle soup over bread.
    6. Sprinkle with Parmesan cheese.
    
    2. Onion and Potato Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -2 cups potatoes, diced
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add potatoes, vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. Serve hot.
    
    3. Creamy Onion Soup
    
    Ingredients:
    
    -1 large onion, chopped
    -3 cups vegetable broth
    -1 cup milk
    -1/4 teaspoon black pepper
    -1/4 cup all-purpose flour
    -1/2 cup shredded Parmesan cheese
    
    Instructions:
    
    1. In a large pot, sauté onions in butter until golden brown.
    2. Add vegetable broth, milk, and pepper. Bring to a boil.
    3. Reduce heat and simmer for 10 minutes.
    4. In a small bowl, whisk together flour and Parmesan cheese until smooth.
    5. Add to soup and simmer for an additional 5 minutes, or until soup has thickened.
    ```

    你會見到，所有有牛奶嘅食譜都已經被篩走。如果你對乳糖敏感，可能仲想篩走有芝士嘅食譜，所以要講清楚。

    ```python
    
- **Produce a shopping list**. We want to produce a shopping list, considering what we already have at home.

    For this functionality, we could either try to solve everything in one prompt or we could split it up into two prompts. Let's try the latter approach. Here we're suggesting adding an additional prompt, but for that to work, we need to add the result of the former prompt as context to the latter prompt. 

    Locate the part in the code that prints out the result from the first prompt and add the following code below:
    
    ```python
    old_prompt_result = completion.choices[0].text
    prompt = "Produce a shopping list for the generated recipes and please don't include ingredients that I already have."
    
    new_prompt = f"{old_prompt_result} {prompt}"
    messages = [{"role": "user", "content": new_prompt}]
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
    
    # print response
    print("Shopping list:")
    print(completion.choices[0].message.content)
    ```

    Note the following:

    - We're constructing a new prompt by adding the result from the first prompt to the new prompt: 
    
        ```python
        new_prompt = f"{old_prompt_result} {prompt}"
        messages = [{"role": "user", "content": new_prompt}]
        ```

    - We make a new request, but also considering the number of tokens we asked for in the first prompt, so this time we say `max_tokens` is 1200. 

        ```python
        completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
        ```  

        Taking this code for a spin, we now arrive at the following output:

        ```output
        No of recipes (for example, 5: 2
        List of ingredients (for example, chicken, potatoes, and carrots: apple,flour
        Filter (for example, vegetarian, vegan, or gluten-free: sugar
        Recipes:
         or milk.
        
        -Apple and flour pancakes: 1 cup flour, 1/2 tsp baking powder, 1/2 tsp baking soda, 1/4 tsp salt, 1 tbsp sugar, 1 egg, 1 cup buttermilk or sour milk, 1/4 cup melted butter, 1 Granny Smith apple, peeled and grated
        -Apple fritters: 1-1/2 cups flour, 1 tsp baking powder, 1/4 tsp salt, 1/4 tsp baking soda, 1/4 tsp nutmeg, 1/4 tsp cinnamon, 1/4 tsp allspice, 1/4 cup sugar, 1/4 cup vegetable shortening, 1/4 cup milk, 1 egg, 2 cups shredded, peeled apples
        Shopping list:
         -Flour, baking powder, baking soda, salt, sugar, egg, buttermilk, butter, apple, nutmeg, cinnamon, allspice 
        ```
        
- **A word on token length**. We should consider how many tokens we need to generate the text we want. Tokens cost money, so where possible, we should try to be economical with the number of tokens we use. For example, can we phrase the prompt so that we can use less tokens?

   To change tokens used, you can use the `max_tokens` parameter. For example, if you want to use 100 tokens, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=100)
    ```

- **Experimenting with temperature**. Temperature is something we haven't mentioned so far but is an important context for how our program performs. The higher the temperature value the more random the output will be. Conversely the lower the temperature value the more predictable the output will be. Consider whether you want variation in your output or not.

   To alter the temperature, you can use the `temperature` parameter. For example, if you want to use a temperature of 0.5, you would do:

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, temperature=0.5)
    ```

   > 注意，越接近 1.0，輸出就會越多變化。



## 作業

今次作業你可以自由選擇想做咩。

以下有啲建議：

- 改良食譜產生器 app，令佢更好用。可以試下調整 temperature 嘅值，或者改下 prompt，睇下可以做到咩效果。
- 做個「學習夥伴」app。呢個 app 可以答你關於某個主題嘅問題，例如 Python，你可以用 prompt 問「Python 嘅某個主題係咩？」或者「畀我睇某個主題嘅程式碼」等等。
- 歷史機械人，令歷史人物「活現眼前」，叫機械人扮演某個歷史人物，問佢關於佢嘅生平同時代嘅問題。

## 參考答案

### 學習夥伴

- 「你係 Python 語言專家

    建議一個初學者 Python 課堂，格式如下：
    
    格式：
    - 概念：
    - 課堂簡介：
    - 程式練習同答案」

以上係一個起步 prompt，可以試下點用，或者按自己喜好改一改。

### 歷史機械人

以下係啲 prompt 可以用：

- 「你係林肯，請用三句說話介紹自己，用林肯會用嘅語法同詞語回應」
- 「你係林肯，請用林肯會用嘅語法同詞語回應：

   用 300 字講下你最大嘅成就：」

## 知識檢查

temperature 呢個概念有咩用？

1. 控制輸出有幾隨機。
1. 控制回應有幾大。
1. 控制用咗幾多 token。

A: 1

儲存 API key 等機密資料，咩方法好？

1. 寫喺程式碼入面。
1. 寫喺檔案入面。
1. 用環境變數。

A: 3，因為環境變數唔會寫喺程式碼入面，可以由程式碼讀取。



---

**免責聲明**：  
本文件經由 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。我們致力確保翻譯準確，但請注意，自動翻譯可能會出現錯誤或不準確之處。原始語言的文件應被視為具權威性的來源。如涉及重要資訊，建議尋求專業人手翻譯。本翻譯所引致的任何誤解或曲解，我們概不負責。
